In [1]:
!pip install gradio
!pip install segmentation-models-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!cp /content/drive/MyDrive/Colab\ Notebooks/AI\ Builder/aib_cdr.py /content

cp: cannot stat '/content/drive/MyDrive/Colab Notebooks/AI Builder/aib_cdr.py': No such file or directory


In [9]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
from PIL import UnidentifiedImageError
import cv2
import segmentation_models_pytorch as sm
import gradio as gr
from aib_cdr import calculate_cdr

In [4]:

class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.model = models.resnet50(pretrained=True)

        for param in self.model.parameters():
            param.requires_grad = False
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 3),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.model(x)
        return x
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.model = models.resnet50(pretrained=True)

        for param in self.model.parameters():
            param.requires_grad = False
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 2)
        )

    def forward(self, x):
        x = self.model(x)
        probas = nn.functional.softmax(x, dim=1)
        return x, probas

In [5]:
disc_model_path = '/content/drive/MyDrive/siriraj-eye-dataset/disc_model.pth'
cup_model_path = '/content/drive/MyDrive/siriraj-eye-dataset/cup_model.pth'
quality_model_path = '/content/drive/MyDrive/siriraj-eye-dataset/quality_model.pth'
camtype_model_path = '/content/drive/MyDrive/siriraj-eye-dataset/camtype_model.pth'
dr_model_path = '/content/drive/MyDrive/siriraj-eye-dataset/dr_model.pth'

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

disc_model = sm.Unet('resnet34', classes=2, activation=None)
disc_model.load_state_dict(torch.load(disc_model_path, map_location=device))
disc_model.to(device)
disc_model.eval()

cup_model = sm.Unet('resnet34', classes=2, activation=None)
cup_model.load_state_dict(torch.load(cup_model_path, map_location=device))
cup_model.to(device)
cup_model.eval()

quality_model = Net1().to(device)
quality_model.load_state_dict(torch.load(quality_model_path,map_location=device))
quality_model.to(device)
quality_model.eval()

camtype_model = models.resnet50(pretrained=True)
num_features = camtype_model.fc.in_features
camtype_model.fc = nn.Linear(num_features, 1)
camtype_model.load_state_dict(torch.load(camtype_model_path,map_location=device))
camtype_model.to(device)
camtype_model.eval()

dr_model = Net2().to(device)
dr_model.load_state_dict(torch.load(dr_model_path,map_location=device))
dr_model.to(device)
dr_model.eval()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Net2(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): 

In [12]:
def model_interface(image):
    image = Image.fromarray(image.astype('uint8'), 'RGB')
    image = image.resize((512, 512))
    image_np = np.array(image)
    image_tensor = transform(image).unsqueeze(0).to(device)

    cdr = calculate_cdr(image_tensor, disc_model, cup_model, device)
    cdr = round(cdr,2)

    quality_probs = quality_model(image_tensor)
    quality_probs = quality_probs.flatten()
    quality_dict = {"Image Quality : ACCEPTABLE": quality_probs[0].item(), "Image Quality : GOOD": quality_probs[1].item(),"Image Quality : POOR" : quality_probs[2].item()}

    camtype_pred = camtype_model(image_tensor)
    camtype_pred = camtype_pred.item()

    camtype = "Eidon" if camtype_pred == 0 else "Nidek"

    dr_grading = dr_model(image_tensor)
    dr_grading = dr_grading[1].flatten()
    dr_dict = {"DR : Negative": dr_grading[0].item(), "DR : Positive": dr_grading[1].item()}

    disc_pred = disc_model(image_tensor)
    disc_mask = (disc_pred.argmax(dim=1) > 0.5).squeeze().cpu().numpy()
    disc_contours, _ = cv2.findContours(disc_mask.astype('uint8'), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    image_np1 = np.array(image)
    image_np1 = cv2.drawContours(image_np1, disc_contours, -1, (0, 255, 0), 2)

    cup_pred = cup_model(image_tensor)
    cup_mask = (cup_pred.argmax(dim=1) > 0.5).squeeze().cpu().numpy()
    cup_contours, _ = cv2.findContours(cup_mask.astype('uint8'), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    image_np2 = np.array(image)
    image_np2 = cv2.drawContours(image_np2, cup_contours, -1, (0, 0, 255), 2)

    return quality_dict, camtype, dr_dict, image_np1 , image_np2,cdr

iface = gr.Interface(
    fn=model_interface,
    inputs=gr.inputs.Image(),
    outputs=[
        gr.outputs.Label(num_top_classes=3, label="Image Quality"),
        gr.outputs.Textbox(label="Camera Type"),
        gr.outputs.Label(num_top_classes=2, label="DR Grading"),
        gr.outputs.Image(type='numpy', label="Disc Mark"),
        gr.outputs.Image(type='numpy', label="Cup Mark"),
        gr.outputs.Textbox(label="Cup-to-disc Ratio")
    ]
)

iface.launch(debug=True)


/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:259: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:262: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:197: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:200: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  super().__init__(num_top_classes=num_top_classes, type=type, label=label)
/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import yo

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
